<a href="https://colab.research.google.com/github/ShivamParihar/project_gutenberg_character_generation/blob/master/project_gutenberg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [5]:
import numpy
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


In [0]:
filename = "/content/gdrive/My Drive/ml_codes/project_gutenberg/new_dataset.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()

In [0]:
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [8]:
nchars = len(raw_text)
n_vocab = len(chars)
print(nchars)
print(n_vocab)

144321
43


In [9]:
seq_length = 100
dataX = []
dataY = []
for i in range(0, nchars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print(n_patterns)

144221


In [7]:
print(dataX[0])

[19, 24, 17, 32, 36, 21, 34, 1, 25, 10, 1, 20, 31, 39, 30, 1, 36, 24, 21, 1, 34, 17, 18, 18, 25, 36, 9, 24, 31, 28, 21, 0, 0, 17, 28, 25, 19, 21, 1, 39, 17, 35, 1, 18, 21, 23, 25, 30, 30, 25, 30, 23, 1, 36, 31, 1, 23, 21, 36, 1, 38, 21, 34, 41, 1, 36, 25, 34, 21, 20, 1, 31, 22, 1, 35, 25, 36, 36, 25, 30, 23, 1, 18, 41, 1, 24, 21, 34, 1, 35, 25, 35, 36, 21, 34, 1, 31, 30, 1, 36]


In [8]:
print(dataY[0])

24


In [0]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))

In [0]:
# normalize
X = X / float(n_vocab)

In [0]:
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [12]:
!pip install keras==2.2.5

In [13]:
!pip install tensorflow==2.0.0

In [0]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [0]:
# define the checkpoint
filepath="/content/gdrive/My Drive/ml_codes/project_gutenberg/weights_improvement.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [16]:
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Train on 144221 samples
Epoch 1/20
144128/144221 [============================>.] - ETA: 0s - loss: 2.9772
Epoch 00001: loss improved from inf to 2.97703, saving model to /content/gdrive/My Drive/ml_codes/project_gutenberg/weights_improvement.hdf5
144221/144221 [==============================] - 887s 6ms/sample - loss: 2.9770
Epoch 2/20
144128/144221 [============================>.] - ETA: 0s - loss: 2.7753
Epoch 00002: loss improved from 2.97703 to 2.77529, saving model to /content/gdrive/My Drive/ml_codes/project_gutenberg/weights_improvement.hdf5
144221/144221 [==============================] - 879s 6ms/sample - loss: 2.7753
Epoch 3/20
144128/144221 [============================>.] - ETA: 0s - loss: 2.6696
Epoch 00003: loss improved from 2.77529 to 2.66956, saving model to /content/gdrive/My Drive/ml_codes/project_gutenberg/weights_improvement.hdf5
144221/144221 [==============================] - 900s 6ms/sample - loss: 2.6696
Epoch 4/20
144128/144221 [============================>.

In [13]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

# load the network weights
filename = "/content/gdrive/My Drive/ml_codes/project_gutenberg/weights_improvement.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [17]:
import sys

# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print(''.join([int_to_char[value] for value in pattern]))
# generate characters
for i in range(1000):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
t used up.'

'but what happens when you come to the beginning again?' alice ventured
to ask.

'suppo
n, and that ' said the monke, shthing to the jury. and the horp ofs oi ae in her hoed an ohce, and the cotmdus sasd thin she was sollling to fnr the was to ald to the kat has hasden teen the was so aedin in the hadd.
'nh whuh the qerten tf thin ' 
'tha sar of the dorsonse ' said the monke, shthing to alice an in ouoert no an sfeer. ''than sou dould be io deleed to thu woine you dan't tiln ' said alice, 'and thet so lete heve thet lo beter ai afriee ' 
'i whnh y uhe kart ri gare to tou then ' sail alice, and she hurph of the word whu would be ailngd to the karth thse the whst on begngng, and saed to the katthen in the horg, and the whrt ondel litele the had noet war to sar the qoeen saed to the katthrn in the court, and taede  she fatt round the saie the sar of thing she whst sadd to the katter. 'no she sert woud wou'd nott ' shought alice, 'i wou'o  aall the horn of theng sheng so